In [1]:
!pip install nmslib

     |████████████████████████████████| 13.0MB 303kB/s 
     |████████████████████████████████| 296kB 44.7MB/s 


In [2]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 217.8MB 57kB/s 
     |████████████████████████████████| 204kB 41.1MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.5-py2.py3-none-any.whl size=218257927 sha256=c35813e1c19f9bb75542386b495ca5cb5e83c9317c9a9be1583c4dfe5a6849dc
  Stored in directory: /root/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 31 not upgraded.
Need to get 35.8 MB of archives.
After this operation, 140 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading

In [0]:
# Let's import the libraries we will need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [0]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [0]:
data_path = '/content/drive/My Drive/Questions.csv'
max_user_id = 7045028

In [6]:
df = pd.read_csv(data_path, encoding="ISO8859-1")
df = df[['Id', 'Title', 'Body']]
df = df.assign(nId=lambda x: x.Id + max_user_id)
df = df.drop(['Id'], axis=1)

# df = df.assign(Text=lambda x: x.Title + x.Body)

df.head(5)

,Title,Body,nId
0,SQLStatement.execute() - multiple queries in o...,<p>I've written a database generation script i...,7045108
1,Good branching and merging tutorials for Torto...,<p>Are there any really good tutorials explain...,7045118
2,ASP.NET Site Maps,<p>Has anyone got experience creating <strong>...,7045148
3,Function for creating color wheels,<p>This is something I've pseudo-solved many t...,7045208
4,Adding scripting functionality to .NET applica...,<p>I have a little game written in C#. It uses...,7045288


In [7]:
df.shape

(1264216, 3)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    ngram_range=(1, 2))
X_tfidf = tfidf.fit_transform(df['Title'])
X_tfidf

<1264216x2122845 sparse matrix of type '<class 'numpy.float64'>'
	with 19088550 stored elements in Compressed Sparse Row format>

In [12]:
import nmslib

index = nmslib.init(method='hnsw', space='cosinesimil_sparse', data_type=nmslib.DataType.SPARSE_VECTOR)
index.addDataPointBatch(X_tfidf)

1264216

In [0]:
index_time_params = {'M': 30, 'indexThreadQty': 8, 'efConstruction': 100, 'post' : 0}
index.createIndex(index_time_params)

In [0]:
nbrs = index.knnQueryBatch(X_tfidf, k = 10, num_threads = 8)

In [28]:
df.iloc[nbrs[1][0]][['nId']].to_numpy().reshape(10,)

array([18966058, 40364608, 13708008, 30403118, 11453198, 31125108,
       47094768, 17273208, 10005718, 24259698], dtype=int64)

In [55]:
from tqdm import tqdm
n_nbrs = {}
for nbr in tqdm(nbrs):
    arr = nbr[0]
    arr = df.iloc[arr][['nId']].to_numpy().reshape(10,)
    n_nbrs[arr[0]] = (arr[1:10], nbr[1][1:10])

100%|██████████| 1264216/1264216 [27:37<00:00, 762.64it/s]


In [53]:
df.iloc[nbrs[1][0]][['nId']]#.to_numpy().reshape(10,)

,nId
331440,18966058
1007509,40364608
171537,13708008
684043,30403118
106228,11453198
706816,31125108
1260222,47094768
278960,17273208
65689,10005718
493761,24259698


In [57]:
n_nbrs[18966058]

(array([13708008, 30403118, 47094768, 17273208, 38379108, 41365668,
        20442568, 11556268, 39515988], dtype=int64),
 array([0.96196026, 0.96255857, 0.9641801 , 0.9642187 , 0.96647877,
        0.96679944, 0.96706927, 0.96726704, 0.96742153], dtype=float32))

In [0]:
import pickle
with open('neighbors-10.pickle', 'wb') as handle:
    pickle.dump(n_nbrs, handle)

In [59]:
os.path.getsize('neighbors-10.pickle')

220253653

In [0]:
with open('neighbors-10.pickle', 'rb') as readin:
    b = pickle.load(readin)

In [61]:
b[18966058]

(array([13708008, 30403118, 47094768, 17273208, 38379108, 41365668,
        20442568, 11556268, 39515988]),
 array([0.96196026, 0.96255857, 0.9641801 , 0.9642187 , 0.96647877,
        0.96679944, 0.96706927, 0.96726704, 0.96742153], dtype=float32))

In [0]:
# from sklearn.neighbors import LSHForest

# lshf = LSHForest(n_neighbors=10)
# lsdf.fit(X_tfidf)

In [0]:
# encodings = df.apply(lambda x: [x.Id, model.encode(x.Title)], axis=1, result_type='expand')

In [0]:
# feature_vec = model.encode(df['Title'].tolist(), show_progress_bar=True, convert_to_numpy=True)